In [214]:
import urllib
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn import svm

In [4]:
%matplotlib inline

In [197]:
airlines = pd.read_csv('/Users/wangzongwei/Desktop/cse258/flight-delays/airlines.csv')
airports = pd.read_csv('/Users/wangzongwei/Desktop/cse258/flight-delays/airports.csv')
flights = pd.read_csv('/Users/wangzongwei/Desktop/cse258/flight-delays/flights.csv', low_memory=False)

In [286]:
D = 60

In [198]:
flights.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
scheduled_mon = [0]*12
for i in range(1,13):
    scheduled_mon[i-1] = flights[flights['MONTH']==i].count()['SCHEDULED_DEPARTURE']

In [66]:
print 'Cancelled:'
operated_mon = [0]*12
cancelled_mon = [0]*12
n_cancelled = flights[(flights['CANCELLED']==1)].count()['CANCELLED']
for i in range(1,13):
    mon_n_cancelled = flights[(flights['CANCELLED']==1)&(flights['MONTH']==i)].count()['CANCELLED']
    cancelled_mon[i-1] = mon_n_cancelled
    operated_mon[i-1] = scheduled_mon[i-1]-mon_n_cancelled
    ratio1 = mon_n_cancelled*1.0/n_cancelled*100
    ratio2 = mon_n_cancelled*1.0/scheduled_mon[i-1]*100
    print 'Mon='+str(i)+' '+str(mon_n_cancelled)+' '+str(ratio1)+'%'+' '+str(scheduled_mon[i-1])+' '+str(ratio2)+'%'
ratio3 = n_cancelled*1.0/sum(scheduled_mon)*100
print 'Total: '+str(n_cancelled)+' '+str(sum(scheduled_mon))+' '+str(ratio3)+'%'

Cancelled:
Mon=1 11982 13.3305148859% 469968 2.54953528751%
Mon=2 20517 22.8260869565% 429191 4.78038915075%
Mon=3 11002 12.2402207289% 504312 2.18158600232%
Mon=4 4520 5.0287036625% 485151 0.931668696962%
Mon=5 5694 6.33483156068% 496993 1.14569018075%
Mon=6 9120 10.1464109296% 503897 1.80989368859%
Mon=7 4806 5.34689154911% 520718 0.922956379461%
Mon=8 5052 5.62057763339% 510536 0.989548239497%
Mon=9 2075 2.30853099551% 464946 0.446288386178%
Mon=10 2454 2.73018557252% 486165 0.504766900127%
Mon=11 4599 5.11659472209% 467972 0.982751104767%
Mon=12 8063 8.97045080326% 479230 1.6824906621%
Total: 89884 5819079 1.54464306121%


In [87]:
cancelled_dow = [0]*7
for i in range(1,8):
    cancelled_dow[i-1] = flights[(flights['CANCELLED']==1)&(flights['DAY_OF_WEEK']==i)].count()['CANCELLED']
    ratio = cancelled_dow[i-1]*1.0/n_cancelled*100
    print 'day of week='+str(i)+' '+str(cancelled_dow[i-1])+' '+str(ratio)+'%'

day of week=1 21073 23.4446620088%
day of week=2 15072 16.7682791153%
day of week=3 10729 11.9364959281%
day of week=4 12291 13.6742913088%
day of week=5 8805 9.79595923635%
day of week=6 8749 9.7336567131%
day of week=7 13165 14.6466556896%


In [111]:
departure_delay = flights[(flights['DEPARTURE_DELAY']>D)].count()['DEPARTURE_DELAY']
arrival_delay = flights[(flights['ARRIVAL_DELAY']>D)].count()['ARRIVAL_DELAY']
duo_delay = flights[(flights['ARRIVAL_DELAY']>D)&(flights['DEPARTURE_DELAY']>D)].count()['ARRIVAL_DELAY']

In [99]:
print 'Arrival Delay > '+str(D)+' min (Monthly):'
arr_delayed_mon = [0]*12
# dep_delayed_mon = [0]*12
month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for i in range(1,13):
    mon_arrival_delay = flights[(flights['ARRIVAL_DELAY']>D)&(flights['MONTH']==i)].count()['ARRIVAL_DELAY']
#     mon_departure_delay = flights[(flights['DEPARTURE_DELAY']>5)&(flights['MONTH']==i)].count()['DEPARTURE_DELAY']
    arr_delayed_mon[i-1] = mon_arrival_delay
#     dep_delayed_mon[i-1] = mon_departure_delay
    ratio1 = mon_arrival_delay*1.0/arrival_delay*100
    ratio2 = mon_arrival_delay*1.0/operated_mon[i-1]*100
#     r = mon_departure_delay*1.0/departure_delay*100
    print str(month[i-1])+' '+str(mon_arrival_delay)+' '+str(ratio1)+'% '+str(operated_mon[i-1])+' '+str(ratio2)+'%'
ratio3 = arrival_delay*1.0/sum(operated_mon)*100
print 'Total: '+str(arrival_delay)+' '+str(sum(operated_mon))+' '+str(ratio3)+'%'

Arrival Delay > 30 min (Monthly):
Jan 56793 8.9763346436% 457986 12.4005973982%
Feb 57601 9.10404190315% 408674 14.0946084165%
Mar 55275 8.73640937131% 493310 11.2049218544%
Apr 46346 7.32514932108% 480631 9.64274048074%
May 54280 8.57914609995% 491299 11.0482618528%
Jun 73381 11.5981267495% 494777 14.8311259416%
Jul 66687 10.540116359% 515912 12.9260416505%
Aug 57801 9.13565261097% 505484 11.4347832968%
Sep 33816 5.34473847671% 462871 7.30570720568%
Oct 32374 5.11682527339% 483711 6.69283931935%
Nov 39658 6.26808725188% 463373 8.55854786533%
Dec 58685 9.27537193949% 471167 12.4552441067%
Total: 632697 5729195 11.0433839309%


In [103]:
print 'Arrival Delay > '+str(D)+' min (Seasonally):'
season = ['Spring', 'Summer', 'Fall', 'Winter']
arr_delayed_season = [0]*4
operated_season = [0]*4
j = 0
for i in range(12):
    arr_delayed_season[j] += arr_delayed_mon[(i+2)%12]
    operated_season[j] += operated_mon[(i+2)%12]
    if i == 2 or i == 5 or i == 8:
        j += 1
for i in range(4):
    ratio1 = arr_delayed_season[i]*1.0/operated_season[i]*100
    ratio2 = operated_season[i]*1.0/sum(operated_season)*100
    print season[i]+' '+str(arr_delayed_season[i])+' '+str(operated_season[i])+' '+str(ratio1)+'% '+str(ratio2)+'%'

Arrival Delay > 30 min (Seasonally):
Spring 155901 1465240 10.639963419% 25.574971702%
Summer 197869 1516173 13.0505555764% 26.463979669%
Fall 105848 1409955 7.50718994578% 24.6100019287%
Winter 173079 1337827 12.9373229872% 23.3510467003%


In [104]:
arr_delayed_dow = [0]*7
dow = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
for i in range(1,8):
    arr_delayed_dow[i-1] = flights[(flights['ARRIVAL_DELAY']>D)&(flights['DAY_OF_WEEK']==i)].count()['ARRIVAL_DELAY']
    ratio = arr_delayed_dow[i-1]*1.0/arrival_delay*100
    print str(dow[i-1])+' '+str(arr_delayed_dow[i-1])+' '+str(ratio)+'%'

Mon 102843 16.2547001171%
Tue 91715 14.4958803345%
Wed 90175 14.2524778844%
Thu 101019 15.9664104619%
Fri 94214 14.8908561286%
Sat 64701 10.2262220305%
Sun 88030 13.9134530431%


In [112]:
print 'departure delayed: '+str(departure_delay)
print 'arrival delayed: '+str(arrival_delay)
ratio1 = duo_delay*1.0/arrival_delay*100
print 'departure & arrival delayed: '+str(duo_delay)+' '+str(ratio1)+'%'
ratio2 = flights[(flights['ARRIVAL_DELAY']>D)&(flights['DEPARTURE_DELAY']<=D)].count()['ARRIVAL_DELAY']*1.0/arrival_delay*100
print 'departure on time & arrival delayed: '+str(flights[(flights['ARRIVAL_DELAY']>D)&(flights['DEPARTURE_DELAY']<=D)].count()['ARRIVAL_DELAY'])+' '+str(ratio2)+'%'

departure delayed: 638370
arrival delayed: 632697
departure & arrival delayed: 526984 83.2916862258%
departure on time & arrival delayed: 105713 16.7083137742%


In [174]:
airlines.head()

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways


In [126]:
airlines_code = {}
for i in range(airlines.shape[0]):
    airlines_code[airlines['IATA_CODE'][i]] = airlines['AIRLINE'][i]

In [139]:
arr_delayed_airline = {}
scheduled_airline = {}
cancelled_airline = {}
operated_airline = {}
for key in airlines_code:
    arr_delayed_airline[key] = flights[(flights['ARRIVAL_DELAY']>D)&(flights['AIRLINE']==key)].count()['ARRIVAL_DELAY']
    scheduled_airline[key] = flights[flights['AIRLINE']==key].count()['SCHEDULED_DEPARTURE']
    cancelled_airline[key] = flights[(flights['AIRLINE']==key)&(flights['CANCELLED']==1)].count()['CANCELLED']
    operated_airline[key] = scheduled_airline[key] - cancelled_airline[key]

In [159]:
delayed_airline = []
for key in arr_delayed_airline:
    delayed_airline.append((arr_delayed_airline[key], key))
delayed_airline.sort(reverse=True)
for t in delayed_airline:
    ratio1 = operated_airline[t[1]]*1.0/sum(operated_airline.values())*100
    ratio2 = t[0]*1.0/operated_airline[t[1]]*100
    print t[1]+': '+str(ratio1)+'% '+str(t[0])+' '+str(ratio2)+'% '+airlines_code[t[1]]

WN: 21.744974643% 134288 10.7791544792% Southwest Airlines Co.
AA: 12.4810728209% 76679 10.7233608134% American Airlines Inc.
EV: 9.71770030519% 67513 12.1263556451% Atlantic Southeast Airlines
DL: 15.2212832693% 67496 7.73986104119% Delta Air Lines Inc.
UA: 8.88693786823% 66321 13.0258273593% United Air Lines Inc.
OO: 10.0955369821% 65631 11.3471290282% Skywest Airlines Inc.
MQ: 4.88038895517% 39167 14.0078753393% American Eagle Airlines Inc.
B6: 4.58654313564% 37339 14.2096570411% JetBlue Airways
NK: 2.01380822262% 22192 19.2346695558% Spirit Air Lines
US: 3.39747556158% 19766 10.1547408656% US Airways Inc.
F9: 1.57523002795% 14902 16.5122772804% Frontier Airlines Inc.
AS: 2.99958371115% 11194 6.5137443847% Alaska Airlines Inc.
VX: 1.07116270261% 7059 11.5025501475% Virgin America
HA: 1.32830179458% 3150 4.1392360153% Hawaiian Airlines Inc.


In [176]:
airports.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [170]:
airports_name = {}
for i in range(airports.shape[0]):
    airports_name[airports['IATA_CODE'][i]] = (airports['AIRPORT'][i], airports['STATE'][i]) 

In [161]:
arr_delayed_airport = {}
arr_flight_vol_airport = {}
for key in airports_name:
    arr_delayed_airport[key] = flights[(flights['ARRIVAL_DELAY']>D)&(flights['DESTINATION_AIRPORT']==key)].count()['ARRIVAL_DELAY']
    operated = flights[flights['DESTINATION_AIRPORT']==key].count()['SCHEDULED_DEPARTURE']-flights[(flights['DESTINATION_AIRPORT']==key)&(flights['CANCELLED']==1)].count()['CANCELLED']
    arr_flight_vol_airport[key] = operated

In [173]:
delayed_airport1 = []
delayed_airport2 = []
delayed_airport3 = []
for key in arr_delayed_airport:
    ratio = arr_delayed_airport[key]*1.0/arr_flight_vol_airport[key]*100
    delayed_airport1.append((arr_delayed_airport[key], arr_flight_vol_airport[key], ratio, key))
    delayed_airport2.append((arr_flight_vol_airport[key], arr_delayed_airport[key], ratio, key))
    delayed_airport3.append((ratio, arr_flight_vol_airport[key], arr_delayed_airport[key], key))
delayed_airport1.sort(reverse=True)
delayed_airport2.sort(reverse=True)
delayed_airport3.sort(reverse=True)
i = 0
print 'Ordered by arrival delay:'
for t in delayed_airport1:
    print t[-1]+': '+str(t[0])+' '+str(t[1])+' '+str(t[2])+'% '+airports_name[t[-1]][0]+', '+airports_name[t[-1]][1]
    i += 1
    if i == 5:
        break
i = 0
print 'Ordered by flight volumn:'
for t in delayed_airport2:
    print t[-1]+': '+str(t[0])+' '+str(t[1])+' '+str(t[2])+'% '+airports_name[t[-1]][0]+', '+airports_name[t[-1]][1]
    i += 1
    if i == 5:
        break
i = 0
print 'Ordered by ratio:'
for t in delayed_airport3:
    print t[-1]+': '+str(t[0])+'% '+str(t[1])+' '+str(t[2])+' '+airports_name[t[-1]][0]+', '+airports_name[t[-1]][1]
    i += 1
    if i == 5:
        break

Ordered by arrival delay:
ORD: 37817 276633 13.6704586944% Chicago O'Hare International Airport, IL
ATL: 30982 344189 9.00144978486% Hartsfield-Jackson Atlanta International Airport, GA
DFW: 27210 232833 11.6864877401% Dallas/Fort Worth International Airport, TX
LAX: 24809 192437 12.8920114115% Los Angeles International Airport, CA
DEN: 22976 193700 11.861641714% Denver International Airport, CO
Ordered by flight volumn:
ATL: 344189 30982 9.00144978486% Hartsfield-Jackson Atlanta International Airport, GA
ORD: 276633 37817 13.6704586944% Chicago O'Hare International Airport, IL
DFW: 232833 27210 11.6864877401% Dallas/Fort Worth International Airport, TX
DEN: 193700 22976 11.861641714% Denver International Airport, CO
LAX: 192437 24809 12.8920114115% Los Angeles International Airport, CA
Ordered by ratio:
STC: 29.8701298701% 77 23 St. Cloud Regional Airport, MN
ILG: 23.1578947368% 95 22 Wilmington Airport, DE
MQT: 22.1476510067% 298 66 Sawyer International Airport, MI
OTH: 21.2686567164

In [182]:
# flight_route = {}
# for key in airports_name:
#     for k in airports_name.keys():
#         if key is not k:
#             flight_route[key+k] = flights[(flights['ARRIVAL_DELAY']>D)&(flights['DESTINATION_AIRPORT']==k)&(flights['ORIGIN_AIRPORT']==key)].count()['ARRIVAL_DELAY']

In [188]:
sample_flights = pd.read_csv('/Users/wangzongwei/Desktop/cse258/flight-delays/sample_flights.csv', low_memory=False, header=None)

In [202]:
sample_flights.columns = ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', \
                          'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', \
                          'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', \
                          'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', \
                          'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', \
                          'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']

In [255]:
sample_flights.tail()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
9995,2015,6,8,1,AA,1312,N3AUAA,PHX,JFK,1235,...,2024.0,-12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
9996,2015,2,4,3,NK,630,N620NK,ORD,LGA,1759,...,2147.0,41.0,0,0,NaN,41.0,0.0,0.0,0.0,0.0
9997,2015,5,14,4,UA,1090,N69824,IAD,DEN,1225,...,1420.0,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
9998,2015,6,19,5,AS,729,N583AS,IAD,SEA,1840,...,2201.0,12.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
9999,2015,2,10,2,US,849,N703UW,ATL,PHL,1945,...,2133.0,-10.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [295]:
X = []
y = []
airlines_id = dict(zip(airlines_code.keys(),range(len(airlines_code))))
airports_id = dict(zip(airports_name.keys(),range(len(airports_name))))
for i in range(sample_flights.shape[0]):
    if sample_flights['CANCELLED'][i] or sample_flights['DIVERTED'][i]: 
        continue
    feat = [1]
    f_mon = [0]*12
    f_mon[sample_flights['MONTH'][i]-1] = 1
    feat += f_mon
    f_dow = [0]*7
    f_dow[sample_flights['DAY_OF_WEEK'][i]-1] = 1
    feat += f_dow
    f_airlines = [0]*len(airlines_code)
    f_airlines[airlines_id[sample_flights['AIRLINE'][i]]] = 1
    feat += f_airlines
    f_airports = [0]*len(airports_name)
    if sample_flights['DESTINATION_AIRPORT'][i] not in airports_id:
        continue
    f_airports[airports_id[sample_flights['DESTINATION_AIRPORT'][i]]] = 1
    feat += f_airports
    feat += sample_flights['DEPARTURE_DELAY'][i]>D
    X.append(feat)
    score = 0
    if sample_flights['ARRIVAL_DELAY'][i]>D:
        score = 1
    elif sample_flights['ARRIVAL_DELAY'][i]<D:
        score = -1
    y.append(score)

In [297]:
X_train = X[:int(len(X)/2)]
X_test = X[int(len(X)/2):]
y_train = y[:int(len(y)/2)]
y_test = y[int(len(y)/2):]

In [299]:
clf = svm.SVC(C=1.0)
clf.fit(X_train, y_train)
test_predictions = clf.predict(X_test)

In [302]:
# # for binary classification 
# TP = sum([y_test[i] and test_predictions[i] for i in range(len(y_test))])
# TN = sum([not y_test[i] and not test_predictions[i] for i in range(len(y_test))])
# FN = sum([y_test[i] and not test_predictions[i] for i in range(len(y_test))])
# FP = sum([not y_test[i] and test_predictions[i] for i in range(len(y_test))])
# TPR = TP*1.0/(TP+FN)
# TNR = TN*1.0/(TN+FP)
# BER = 1-0.5*(TPR+TNR)
# print BER

In [312]:
tri_accuracy = sum([y_test[i] == test_predictions[i] for i in range(len(y_test))])*1.0/len(y_test)

0.98531048297351431

In [316]:
bc = sum([y_test[i]==-1 and test_predictions[i]!=-1 for i in range(len(y_test))])
abc = sum([y_test[i]==-1 for i in range(len(y_test))])
df = sum([y_test[i]==0 and test_predictions[i]!=0 for i in range(len(y_test))])
ndef = sum([y_test[i]==0 for i in range(len(y_test))])
gh = sum([y_test[i]==1 and test_predictions[i]!=1 for i in range(len(y_test))])
ghi = sum([y_test[i]==1 for i in range(len(y_test))])
tri_ber = (bc*1.0/abc+df*1.0/ndef+gh*1.0/ghi)*1.0/3
print tri_ber

0.384187533829
